# 1.INTRODUCTION
 

This notebook belongs to Adnan Kerem AKSOY and Omer Cem TABAR, all copyrights reserved.

In [ ]:
#Checking whether cuda is properly existed within the device
import torch
dev = 'cuda' if torch.cuda.is_available() == True else 'cpu'
if torch.cuda.is_available() == True:
    print("ok")
device =  torch.device(dev)

# 2.DATASET DOWNLOAD AND PREPROCESSING

In [ ]:
#Import all required libraried to proceed
import numpy as np
import pandas as pd
import math as m
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import time as time
from IPython import display
from torchvision import transforms
from numpy import genfromtxt
import matplotlib.pyplot as plt
import os
from datetime import datetime
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, TensorDataset

## 2.1 HOW TO PROCESS IMAGES INTO LIST

In [ ]:
#Load the images into numpy array
def load_images(folder):
    images_list = []
    for filename in os.listdir(folder):
        img = mpimg.imread(os.path.join(folder, filename))
        if img is not None:
            images_list.append(img)
    return np.array(images_list)
images_list = load_images(r'data/anime_faces_data')
print(type(images_list))
print(images_list.shape)

## 2.3 FORMAT CHECK AND PROCESSING FOR GANs

In [ ]:
#Control whether all the images are at size (64*64*3)
control_list=[]
counter = 0
for i in images_list:
    if i.shape != (64, 64, 3):
        control_list.append(i)
print(len(control_list))

In [ ]:
#See the image representation in order to check whether it is successfully loaded(randomly chosen)
num = random.randint(0, 21551)
print("Image number that is randomly generated", num)
print("Total number of images within the list", len(images_list))
print("ndarray representation of the randomly chosen image",images_list[num])
print("class type of the randomly chosen image",type(images_list[num]))
plt.imshow(images_list[num])

print("\n\n\n-----------\n\n\n",images_list[num].shape)

In [ ]:
#Checking one of the images in order to analyze the range of each pixel value
#R,G,B values are taken into consideration seperately.
img = mpimg.imread('data/anime_faces_data/'+str(num)+'.png')

#It is observed that there is no outlier value that exceed the range [0,1]
UnNormalizedCounter = 0
for pixelX in range(0,img.shape[0]):
  for pixelY in range(0,img.shape[1]):
    for channelNum in range(0,img.shape[2]):
      if img[pixelX][pixelY][channelNum] < 0 and img[pixelX][pixelY][channelNum] > 1:
        UnNormalizedCounter = UnNormalizedCounter + 1
if UnNormalizedCounter > 0:
  print("Values are not normalized")
else:
  print("Values are normalized")

################################################ NORMALIZATION NOTE #########################################################
#There is no need for the normalization of the pixel values.
#There exist 3 different values for each pixel(denoted as R(Red), G(Green) and B(Blue) values)
#and each of the pixel value stands between the range [0,1] which states that values are already normalized.
#Hence there is no need for normalization for the pixel values in the images before feeding them to the GAN(Generative Adversarial Network)
#############################################################################################################################


In [ ]:
my_images = images_list
tensor_images = torch.Tensor(my_images)
# we are turning our data to tensors to create the pytorch dataset
my_dataset = TensorDataset(tensor_images.permute(0, 3, 1, 2),torch.tensor(np.ones(len(images_list))))
my_dataloader = DataLoader(my_dataset)

In [ ]:
print(my_dataset)

In [ ]:
print(tensor_images.shape)
print(tensor_images.permute(0, 3, 1,2))

# 3.GENERATOR AND DISCRIMINATOR

In [ ]:
###Hocanın örneği###
BATCH_SIZE = 64 #64 ile başladık
# batch size equals to 4, we can also use 8 but in some points it creates some noise in the loss funcion graph

dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=os.cpu_count())
# we make our dataloader ready to use in the GAN

## 3.1 GENERATOR IMPLEMENTATION

In [ ]:
class Generator(nn.Module):

    def __init__(self, input_size):

        super().__init__()

        self.input_size = input_size

        self.fc_net = nn.Sequential(
            nn.Linear(input_size, 256*16*16, bias=False),#used to create single layer with (#input size) input and 256*10*32
            nn.BatchNorm1d(256*16*16),
            nn.ReLU()
        )

        self.conv_model = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 5, bias=False, padding=2),
            nn.BatchNorm2d((128)),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 5, stride=2, bias=False, padding=2, output_padding=1),
            nn.BatchNorm2d((64)),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 5, stride=2, bias=False, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        y = self.fc_net(x)
        y = y.reshape((-1,256, 16, 16))
        y = self.conv_model(y)
#         my_generator_images.append(y)
        return y

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.xavier_uniform_(m.weight.data)
    if classname.find('Linear') != -1:
        #nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.xavier_uniform_(m.weight.data)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.2)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
generator = Generator(100)
generator.apply(weights_init)
# assign the weights

In [ ]:
###########################################################
noise = torch.normal(0, 1, [10, 100])
print(noise.shape)
generated_image = generator(noise).detach()
print(generated_image.shape)
###########################################################

plt.imshow(  generated_image.squeeze()[0].permute(1, 2, 0)  )
print(np.array(generated_image.squeeze()[0]).shape)
# using generator to generate a image


## 3.2 DISCRIMINATOR IMPLEMENTATION

In [ ]:
class Discriminator(nn.Module):

    def __init__(self):

        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=1e-1,inplace=True),
            nn.Conv2d(64, 128, 5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=1e-1,inplace=True),
            nn.Conv2d(128, 128, 5, stride=2, padding=2),
            nn.LeakyReLU(negative_slope=1e-1,inplace=True),
            nn.Flatten(),
            nn.Linear(8192,1)
        )


    def forward(self, x):
#         labels_gan = []

        y = self.model(x)
#         my_discriminator_results.append(y)
        return y

In [ ]:
discriminator = Discriminator()
discriminator.apply(weights_init)
# weights have been assigned

In [ ]:
###########################################################
decision = discriminator(generated_image)
###########################################################
# discriminators result to generated image
print(decision)

In [ ]:
cross_entropy = nn.BCEWithLogitsLoss()
Generator_loss_list = []
Discriminator_loss_list = []
# controll lists
def discriminator_loss(real_output, fake_output, device):

    real_loss = cross_entropy(real_output, torch.ones_like(real_output, device=device))
    print("Discriminator Real_Loss:",real_loss)
    fake_loss = cross_entropy(fake_output, torch.zeros_like(fake_output, device=device))
    print("Discriminator Fake_Loss:",fake_loss)
    total_loss = real_loss + fake_loss
    print("Discriminator Total_Loss:", total_loss)
    Discriminator_loss_list.append([real_loss,fake_loss,total_loss])
    # loss function for discriminator
    return total_loss

In [ ]:
def generator_loss(fake_output, device):
    ###########################################################
    gen_loss = cross_entropy(fake_output, torch.ones_like(fake_output, device=device))
    ###########################################################
    print("Generator_Loss:",gen_loss)
    Generator_loss_list.append([gen_loss])
    # loss function for generator
    return gen_loss


## 3.3 TRAINING STEP FUNCTIONS

In [ ]:
#In order to track the generated and corresponding real images we need to save them in each batch and in each epoch
generated_images =[]
real_images = []
predicted_images_list = []
raw_images_list = []

In [ ]:
gen_opt = torch.optim.Adam(generator.parameters(), lr=7e-4)
dis_opt = torch.optim.Adam(discriminator.parameters(), lr=7e-4)
# our generator and discriminator uses ADAM optimizator
# we have used learning rates between 1e-3 to 1e-6
# the most optimized one is 1e-4, this one is also is the one that creates less noise
##toplamda 290(+100+250+75 w 16 batch size and (9 * 10^-7)=learning rate)  epoch oldu. ilk 50 epoch 10^(-4) learning rate, sonraki 20 epoch 5* 10^(-6) ardından 220 epoch 2*10^(-6) learning rate
## en son 100 epoch yine 2*10^(-6) learning rate

In [ ]:
EPOCHS = 50 # we assign 400, we can train it more but our GPU RAM is getting almost full and because of that we
# cannot do the last steps, so we generally use 350 to 360 epochs
noise_dim = 100
num_examples_to_generate = 64
seed = torch.randn([num_examples_to_generate, noise_dim], device=device)

In [ ]:

# control lists
def train_step(images, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt, bool):
    noise = torch.randn([BATCH_SIZE, noise_dim], device=device)
    global predicted_images_list
    global raw_images_list
    generated_images = generator(noise)
    if bool == True: 
        raw_images_list.append(images)
        predicted_images_list.append(generated_images)
#     print(images)
#     print(len(images[0]))
    real_output = discriminator(images)

    fake_output = discriminator(generated_images.detach())
#     argmax_real.append(discriminator(images)) # control lists
#     argmax_fake.append(discriminator(generated_images.detach())) # controll lists


    disc_loss = discriminator_loss(real_output, fake_output, device)
    dis_opt.zero_grad()
    disc_loss.backward()
    dis_opt.step()

    fake_output = discriminator(generated_images)
    gen_loss = generator_loss(fake_output, device)
    gen_opt.zero_grad()
    gen_loss.backward()
    gen_opt.step()
    return gen_loss, disc_loss


In [ ]:
def train(dataloader, epochs, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt):
  gloss = []
  dloss = []
  for epoch in range(epochs):
    start = time.time()
    # training both discriminator and generator simultaneously.
    gen_losses = []
    disc_losses = []
    for image_batch, _ in dataloader:
        image_batch = image_batch.to(device)
        if epochs - epoch <4:
            gen_loss, disc_loss = train_step(image_batch, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt, True)
        else: 
            gen_loss, disc_loss = train_step(image_batch, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt, False)
        gen_losses.append(gen_loss.detach().cpu())
        disc_losses.append(disc_loss.detach().cpu())

    gloss.append(np.mean(gen_losses))
    dloss.append(np.mean(disc_losses))
    #Produce images
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed,False,discriminator)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))



  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed,False,discriminator)
  return gloss, dloss

## 3.4 SAVE OBTAINED IMAGES

In [ ]:

def generate_and_save_images(model, epoch, test_input,boolean,discri):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    model.eval()
    with torch.no_grad():
        predictions = model(test_input).detach().cpu()* 250

    grid = make_grid(predictions, 8).numpy().squeeze().transpose(1, 2, 0)
    print(type(grid))

    plt.imshow(grid.astype(np.uint16))
    plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    if boolean == True:
        # if the boolean is True than in this part we get the predictions and use our discriminator to label them
        # with our labeling, we show the transposed images we have created individually.
        predictions = model(test_input)
#         print(predictions,"\n\n\n")
#         print(len(predictions),"\n\n\n")
        predictions32 = predictions[:32]
#         print("\n\n",predictions32,"\n\n")
#         print(len(predictions32))
#         for predict in predictions32:

        label_prediction=discri(predictions[:32])
        # predictions preparation

        list_label_predictions = []
#         for i in label_prediction:
#             list_label_predictions.append(torch.argmax(i))
            # labeling part

        # as result we returned the predictions and the labels of these predictions that have been assigned by the
#         discriminator

        return list_label_predictions,predictions
    model.train()

## 3.5 TRAINING STEP

In [ ]:
# loading pretrained model before start training
discriminator = torch.load('data\pretrained\discridenemesoftmaxlast1')
generator = torch.load('data\pretrained\generadenemesoftmaxlast1')
import io
import pickle

In [ ]:
##########################################################
generator.to(device)
discriminator.to(device)
gloss, dloss = train(dataloader, EPOCHS, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt)
###########################################;################

In [ ]:

# saving first and second checkpoints generatorloss to a file
with open("generatorloss1", "wb") as fp:   #Pickling
    pickle.dump(gloss, fp)

In [ ]:
# saving first and second checkpoints discriminatorloss to a file
with open("discriminatorloss1", "wb") as fp:   #Pickling
    pickle.dump(dloss, fp)

In [ ]:
plt.imshow(predicted_images_list[1010][30].detach().cpu().permute(1,2,0))
# print(len(raw_images_list))


In [ ]:
plt.imshow(raw_images_list[1010][30].detach().cpu().permute(1,2,0))

In [ ]:
# saving tensor to a file
torch.save(predicted_images_list[1010], 'predicted_images_tensorstart.t')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(predicted_images_list[1010], buffer)

In [ ]:
# saving tensor to a file
torch.save(raw_images_list[1010], 'raw_images_liststart.t')
# Save to io.BytesIO buffer
buffer = io.BytesIO()
torch.save(raw_images_list[1010], buffer)

In [ ]:
raw_images_list[1009].shape

In [ ]:
label_predictions_generate_and_save,predictions_generate_and_save = generate_and_save_images(generator,
                           EPOCHS,
                           seed,True,discriminator)
# with making boolean True, we are extracting the predictions and their labels


In [ ]:
# num = 14
label_list_string=[]
# with this part, we turn our numeric label indicators to real string datatype indicators like we get from the timestamps part.
for num in range(32):

#     for label_name in range(len(mDoppler_labels_WR14_set)):
#         if label_name == label_predictions_generate_and_save[num].detach().cpu().tolist():
#             label_list_string.append(list(mDoppler_labels_WR14_set)[label_name])
#             break


    plt.figure()
    ready_img=predictions_generate_and_save[num].detach().cpu().numpy().transpose(1, 2, 0)
    ready_img_up = ready_img.astype(np.float64)
    plt.imshow(ready_img_up)

#     plt.title(list(mDoppler_labels_WR14_set)[label_name])
    plt.savefig('generated and labeled image raw {}'.format(num))
    # we show the micro doppler maps and their labels that have been assigned by the discriminator individually
    # you can see that our generated micro doppler signals are a little bit noisy
#     print(label_predictions_generate_and_save[num].detach().cpu().tolist())


# 4.GAN EVALUATION METHODS

There are to methods for the evaluation of the Generative Adversarial Network(GAN) Performance Evaluation
In the below part you can observe the Inception Score(IS) and Fretchet ınception Distance(FID) Score measurements for the GAN

Both evaluation method are using Inception Models that depends on the InceptionModelV3
We used the pretrainde model in order to obtain the probabilistic predictions of the classes for both real and generated images
Since the generation of the images are handled iteratively, we saved both generated and corresponding real image in each loss calculation within the discriminator point. By this mean we are calculating the corresponding score from both IS and FID perspective

One problem is that, since the inception model assumes that the images are in the format of 299x299x3 with pixel values [0,255], we upscaled our images from 64x64x3 to 128x128x3 with the help of openCV version 2 with the INTERPOLATION_METHOD = INTER_CUBIC.

Choice of the INTER_CUBIC comes from the idea that we want to increas our pixel sizes from 1x1 to 4x4  without any loss of generality since we want to upscale our image from 64x64 to 128x128.

You can follow the dedicated function for upscaling, InceptionScore and FIDScore calculation below

In [ ]:
#Preperation of the Inception Model for both score calculations
import h5py 
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input


InceptionModel = InceptionV3(include_top=False, pooling='avg', input_shape=(128,128,3))

In [ ]:

pip install h5py

In [ ]:
#Required libraries for the score calculations
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm

from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp

In [ ]:
import tensorflow as tf
import cv2

In [ ]:
tf_list_raw_Last = torch.load('raw_images_tensor1683.t')
tf_list_generated_last = torch.load('predicted_images_tensor1683.t')
# raw_images_list = torch.load('raw_images_liststart.t')
# predicted_images_list = torch.load('predicted_images_tensorstart.t')


In [ ]:
print(raw_images_list.shape)

In [ ]:
#turnin the 100 epoch model's batch to tensoflow to use it inside FID and IS
tf_list_raw_list_last = []
tf_list_generated_list_last = []

for i in range(64):
    img_upsample = cv2.resize(tf_list_generated_last[i].permute(1,2,0).detach().cpu().numpy(), dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    # print(img_upsample.shape)
    img_tf_v1 = tf.convert_to_tensor(img_upsample)
    img_tf_v2 = tf.convert_to_tensor(img_tf_v1)
    tf_list_generated_list_last.append(img_tf_v2)
torhc_list_tensor_generated_last = tf.convert_to_tensor(tf_list_generated_list_last)

for i in range(64):
    img_upsample = cv2.resize(tf_list_raw_Last[i].permute(1,2,0).detach().cpu().numpy(), dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    img_tf_v1 = tf.convert_to_tensor(img_upsample)
    img_tf_v2 = tf.convert_to_tensor(img_tf_v1)
    tf_list_raw_list_last.append(img_tf_v2)
torhc_list_tensor_raw_last = tf.convert_to_tensor(tf_list_raw_list_last)
# print(len(tf_list_1[0][0][0]))
print(torhc_list_tensor_generated_last.shape)
print(torhc_list_tensor_raw_last.shape)

In [ ]:
#turnin the pretrained model's batch to tensoflow to use it inside FID and IS
tf_list_1 = []
tf_list_2 = []

for i in range(64):
    image = predicted_images_list[i].detach().cpu().permute(1,2,0).numpy()
    print(image.shape)
    img_upsample = cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    # print(img_upsample.shape)
    img_tf_v1 = tf.convert_to_tensor(img_upsample)
    img_tf_v2 = tf.convert_to_tensor(img_tf_v1)
    tf_list_1.append(img_tf_v2)
torhc_list_tensor_generated_start = tf.convert_to_tensor(tf_list_1)

for i in range(64):
    img_upsample = cv2.resize(raw_images_list[i].permute(1,2,0).detach().cpu().numpy(), dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    img_tf_v1 = tf.convert_to_tensor(img_upsample)
    img_tf_v2 = tf.convert_to_tensor(img_tf_v1)
    tf_list_2.append(img_tf_v2)
torhc_list_tensor_raw_start = tf.convert_to_tensor(tf_list_2)
# print(len(tf_list_1[0][0][0]))
print(torhc_list_tensor2.shape)
print(torhc_list_tensor1.shape)

In [ ]:
plt.imshow(torhc_list_tensor2[0]) #tensor check

## 4.1 INCEPTION SCORE(IS) EVALUATION

In [ ]:
#The inception score is calculated by first using a pre-trained Inception v3 model to predict the class probabilities for each real and generated image.
def InceptionScoreCalculation(images, number_of_splits):
    #Predict the class probabilites for the images from the pretrained model
    #These prediction reflect conditional probability
    #High quality means low entropy
    predicted_class_probabilites = InceptionModel.predict(images)
    # enumerate splits of images/predictions
    inception_scores = []
    
    #In most of the approaches for the calculation of inception score 
    #it is suggested that splitting the imageset
    #and considering the average inception scores and standard deviations is sufficient
    
    for index in range(number_of_splits):
        #Splitting and taking the conditional probabilities
        indexStart, indexEnd = (index * floor(images.shape[0] / number_of_splits)), (index * floor(images.shape[0] / number_of_splits) + floor(images.shape[0] / number_of_splits))
        conditional_set = predicted_class_probabilites[indexStart:indexEnd]
        #Calculate the probability of the class
        classProbability = expand_dims(conditional_set.mean(axis=0), 0)
        # Calculating KL Divergence with log probabilties
        KLDivergence = conditional_set * (log(conditional_set + 1e-3) - log(classProbability +1e-3))
        #Summing the values of divergences
        sumKLDivergence = KLDivergence.sum(axis=1)
        #Averaging the KL values over sumKLDivergence
        averageKLDivergence = mean(sumKLDivergence)
        #Reverse the log operation
        inceptionScore = exp(averageKLDivergence)
        # store the inception score
        inception_scores.append(inceptionScore)
        #Take the average of the inception score and standard devaiton on images
        inceptionScoreAverage, inceptionScoreStandarDeviation = mean(inception_scores), std(inception_scores)
    return inceptionScoreAverage, inceptionScoreStandarDeviation
 
#Calculate inception score with the given image set
#Do not forget to choose the splits properly according to the image set size
inceptionScoreAverage, inceptionScoreStandarDeviation = InceptionScoreCalculation(torhc_list_tensor_generated_last,64)
print('Average Inception Score', inceptionScoreAverage)
print('Standard Deviation of the Inception Score', inceptionScoreStandarDeviation)

## 4.2 FRETCHET INCEPTION DISTANCE(FID) EVALUATION

In [ ]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data

In [ ]:
############################################# PART FOR THE GAN EVALUATION METRICS ########################################################
#This part corresponds to the objective GAN evaluation metrics which are IS(Inception Score) and FID(Fretchet Inception Distannce)
##########################################################################################################################################
# example of calculating the frechet inception distance in Keras

# calculate frechet inception distance
def FIDScoreCalculation(inceptionModel, image1, image2):
    
    #Calculate the class prediction probabilities for the first image
    classProbabilitiesImage1 = inceptionModel.predict(image1)
    
    #Calculate the class prediction probabilities for the first image
    classProbabilitiesImage2 = inceptionModel.predict(image2)
    
    #Calculate the mean and the covarinces of the images
    meanFirstImage, StandardDeviationFirstImage = classProbabilitiesImage1.mean(axis=0), cov(classProbabilitiesImage1, rowvar=False)
    meanSecondImage, StandardDeviationSecondImage = classProbabilitiesImage2.mean(axis=0), cov(classProbabilitiesImage2, rowvar=False)

    #Calculate the sum of squared differences between means
    sumOfSqauredDistanceMean = numpy.sum((meanFirstImage - meanSecondImage)**2.0)
    
    #Calculate the square root of product between covariances
    meanOfTheCovariances = sqrtm(StandardDeviationFirstImage.dot(StandardDeviationSecondImage))
    
    #Check whether the imaginary numbers are in correct format after the square root
    if iscomplexobj(meanOfTheCovariances):
        meanOfTheCovariances = meanOfTheCovariances.real
    
    #Calculate FID Score
    FIDScore = sumOfSqauredDistanceMean + trace(StandardDeviationFirstImage + StandardDeviationSecondImage - 2.0 * meanOfTheCovariances)
    return FIDScore
 

# fid between images1 and images1
fid = FIDScoreCalculation(InceptionModel, torhc_list_tensor2.numpy(), torhc_list_tensor2.numpy())
print('If the image sets are same FID score will be', fid)
# fid between images1 and images2
fid = FIDScoreCalculation(InceptionModel,torhc_list_tensor2.numpy(), torhc_list_tensor1.numpy())
print('If the image sets are different FID score will be', fid)

# 5.ANALYSIS ON RESULTS

## 5.1 LOSS COMPARISONS - CONVERGENCE

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(gloss, label="Generator loss")
plt.plot(dloss, label="Discriminator loss")
plt.legend()
plt.show()

In [ ]:
import pickle
# reading the loss of two checkpoints
with open('generatorloss', 'rb') as f:
    generatorloss0 = pickle.load(f)
with open('generatorloss1', 'rb') as f:
    generatorloss1 = pickle.load(f)
with open('discriminatorloss', 'rb') as f:
    discriminatorloss0 = pickle.load(f)
with open('discriminatorloss1', 'rb') as f:
    discriminatorloss1 = pickle.load(f)

In [ ]:
# combining losses of two checkpoints
for i in generatorloss1:
    generatorloss0.append(i)
for i in discriminatorloss1:
    discriminatorloss0.append(i)


In [ ]:
# plotting the combined losses
# print(generatorloss0)
# print(generatorloss1)
plt.plot(generatorloss0,c='r',label="Generator Loss")
plt.plot(discriminatorloss0, c= 'b',label="Discriminator Loss")
plt.legend(loc='upper right')